In [1]:
import sys
app_path = r'D:\Archivai-AI'
sys.path.append(app_path)

In [ ]:
from app.src.utils.inference import classify_file_bytes

TsCDA7fBiD4W11XUIHFcTFT6iVLhEVlA
https://cohere-embed-multilingual-archiv.eastus2.models.ai.azure.com


In [ ]:
import torch
import os
import cohere
from app.src.utils.classifier import TextClassifier
from app.src.utils.config import encoder, checkpoint
from app.ocr_functions import extract_text
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient


In [6]:
# Example usage of classify_file_bytes function
file_path = r"D:\Archivai-AI\app\src\utils\Architecture Diagram Example - Multiplayer (Community).jpg"  

with open(file_path, "rb") as f:
    file_bytes = f.read()

result = classify_file_bytes(file_bytes)
print(result)

Cohere API Key: "TsCDA7fBiD4W11XUIHFcTFT6iVLhEVlA"
Cohere Endpoint: "https://cohere-embed-multilingual-archiv.eastus2.models.ai.azure.com"


UnsupportedProtocol: Request URL is missing an 'http://' or 'https://' protocol.

In [8]:
file_path = r"D:\Archivai-AI\app\src\utils\Architecture Diagram Example - Multiplayer (Community).jpg" 

with open(file_path, "rb") as f:
    file_bytes = f.read() 


In [24]:

# Extract text from the file bytes
result = extract_text(file_bytes, url=False)

# Combine all pages into one markdown and one raw text block
full_markdown = "\n\n".join(page.markdown_text for page in result)
full_raw = "\n".join(page.raw_text for page in result)

# Authenticate to Azure Key Vault
credential = DefaultAzureCredential()
keyvault_name = "vaultarchivai"
kv_uri = f"https://{keyvault_name}.vault.azure.net"
keys_client = SecretClient(vault_url=kv_uri, credential=credential)

# load environment variables
import dotenv
dotenv.load_dotenv()

# get Cohere credentials from Azure Key Vault
cohere_api_key = keys_client.get_secret("CO-API-KEY").value
cohere_endpoint = keys_client.get_secret("AZURE-ML-COHERE-EMBED-ENDPOINT").value

# checking api_key and endpoint
print("Cohere API Key:", cohere_api_key)
print("Cohere Endpoint:", cohere_endpoint) 


# making embeddings client 
import requests

headers = {
    "Authorization": f"Bearer {cohere_api_key}",
    "Content-Type": "application/json",
}

data = {
    "input_type": "classification",
    "texts": [full_raw]
}

response = requests.post(cohere_endpoint, headers=headers, json=data)

if response.status_code != 200:
    print("Error:", response.text)
    raise RuntimeError("Failed to get embedding")

embeddings = response.json()["embeddings"]

# Check if embeddings are made
print("Embeddings created:", embeddings[0:10])  

# creating a pytorch tensor
embeddings_tensor = torch.tensor(embeddings, dtype=torch.float32)

# instantiating the model loading the checkpoint
model = TextClassifier()
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # Set to evaluation mode

# Get predictions and confidence scores
with torch.no_grad():
    outputs = model(embeddings_tensor)
    
    # Get class predictions
    _, predicted_classes = torch.max(outputs, 1)

    # Get names of the classes
    classes = encoder.inverse_transform(predicted_classes.cpu().numpy())

    # Get confidence scores (using softmax)
    confidences = torch.nn.functional.softmax(outputs, dim=1)
    confidence_values, _ = torch.max(confidences, 1)

    # Extract single value
    folder = classes[0] if len(classes) > 0 else "unknown"
    accuracy = confidence_values[0].item()
    section = "/SyntaxSquad/"


Cohere API Key: "TsCDA7fBiD4W11XUIHFcTFT6iVLhEVlA"
Cohere Endpoint: "https://cohere-embed-multilingual-archiv.eastus2.models.ai.azure.com"


InvalidSchema: No connection adapters were found for '"https://cohere-embed-multilingual-archiv.eastus2.models.ai.azure.com"'